In [1]:
import numpy as np
import pandas as pd

# Edge

In [2]:
class Edge():
  def __init__(self, fromNode, toNode, weight=0):
    self.fromNode = fromNode
    self.toNode = toNode
    self.weight = float(weight)

  # Adjust the weight of an Edge
  def weight(self, weight):
    self.weight = float(weight)

# VertexNode

In [3]:
class VertexNode():
  def __init__(self, value):
    self.value = value
    self.inNodes = []
    self.outNodes = []
    self.totalIn = 0
    self.totalOut = 0
    self.inwardEdges = []
    self.outwardEdges = []
    self.probDist = []

  # Connect to another VertexNode
  def connect(self, toNode, weight=0):
    if toNode not in self.outNodes:
      self.outNodes.append(toNode)
      self.totalOut += 1
      toNode.inNodes.append(self)
      toNode.totalIn += 1
      edge = Edge(self, toNode, weight)
      self.outwardEdges.append(edge)
      toNode.inwardEdges.append(edge)
    # else:
    #   raise Exception('Already connected!')

  # Print inNodes, outNodes, inwardEdges or outwardEdges of a node
  def printInOut(self, type='Node', dir='In'):
    print('+'*20)
    data = []
    lst = []
    DF = []
    if type == 'Node':
      if dir == 'In':
        lst = self.inNodes
      elif dir == 'Out':
        lst = self.outNodes
      for node in lst:
        data.append(node.value)
      print(dir + 'Nodes of {}'.format(self.value))
      DF = pd.DataFrame(np.array(data), columns=['Node'])
    elif type == 'Edge':
      if dir == 'In':
        lst = self.inwardEdges
      elif dir == 'Out':
        lst = self.outwardEdges
      for edge in lst:
        data.append([edge.fromNode.value, edge.toNode.value, edge.weight])
      print(dir + 'wardEdges of {}'.format(self.value))
      DF = pd.DataFrame(np.array(data), columns=['From', 'To', 'Weight'])
    print(DF)
    print('+'*20)

  # Disconnect a connected VertexNode from another VertexNode
  @staticmethod
  def disconnect(fromNode, toNode):
    if (toNode in fromNode.outNodes) & (fromNode in toNode.inNodes):
      edge = (VertexNode).getEdge(fromNode, toNode)
      if edge is not None:
        fromNode.outNodes.remove(toNode)
        fromNode.totalOut -= 1
        toNode.inNodes.remove(fromNode)
        toNode.totalIn -= 1
        fromNode.outwardEdges.remove(edge)
        toNode.inwardEdges.remove(edge)
      # else:
      #   raise Exception('Edge not found!')
    # else:
    #   raise Exception('Not connected!')

  # Get edge between VertexNodes
  @staticmethod
  def getEdge(fromNode, toNode):
    if (toNode in fromNode.outNodes) & (fromNode in toNode.inNodes):
      for edge in fromNode.outwardEdges:
        if (edge.fromNode is fromNode) & (edge.toNode is toNode):
          return edge
        else:
          return None
    else:
      return None

# Graph

In [4]:
class Graph():
  def __init__(self, nodeList=[]):
    self.nodeList = nodeList

  # Add new VertexNode to the graph
  def add(self, value):
    if self.getVertexNode(value) is None:
      newNode = VertexNode(value)
      self.nodeList.append(newNode)
    # else:
    #   raise Exception('Existed Vertex!')

  # Connect one VertexNode to another
  def connect(self, fromValue, toValue, weight=0):
    fromNode = self.getVertexNode(fromValue)
    toNode = self.getVertexNode(toValue)
    if (fromNode is not None) & (toNode is not None):
      fromNode.connect(toNode, weight)
    # else:
    #   raise Exception('Vertex not found!')

  # Disconnect between 2 VertexNodes
  def disconnect(self, fromValue, toValue):
    fromNode = self.getVertexNode(fromValue)
    toNode = self.getVertexNode(toValue)
    if (fromNode is not None) & (toNode is not None):
      VertexNode.disconnect(fromNode, toNode)
    # else:
    #   raise Exception('Vertex not found!')

  # Remove a VertexNode from the graph
  def remove(self, value):
    node = self.getVertexNode(value)
    if node is not None:
      for inNode in node.inNodes.copy():
        VertexNode.disconnect(inNode, node)
      for outNode in node.outNodes.copy():
        VertexNode.disconnect(node, outNode)
      self.nodeList.remove(node)
    # else:
    #   raise Exception('Vertex not found!')

  # Get a node from the graph
  def getVertexNode(self, value):
    nodeExist = False
    for node in self.nodeList:
      if node.value == value:
        nodeExist = True
        return node
    if not nodeExist:
      return None

  # Get inward or outward Edges
  def getEdges(self, direction='outward'):
    edges = []
    if direction == 'outward':
      for node in self.nodeList:
        edges.extend(node.outwardEdges)
    elif direction == 'inward':
      for node in self.nodeList:
        edges.extend(node.inwardEdges)
    return edges

  # Print graph info
  def printGraph(self):
    nodes = self.nodeList
    edges = self.getEdges()
    nodeData = []
    edgeData = []
    print('='*40)
    print('Vertices:')
    for node in nodes:
      nodeData.append([node.value, node.totalIn, node.totalOut])
    nodeDF = pd.DataFrame(np.array(nodeData), columns=['Vertex', 'In', 'Out'])
    print(nodeDF)
    print('-'*20)
    print('Edges:')
    for edge in edges:
      edgeData.append([edge.fromNode.value, edge.toNode.value, edge.weight])
    edgeDF = pd.DataFrame(np.array(edgeData), columns=['From', 'To', 'Weight'])
    print(edgeDF)
    print('='*40)


# Testing Graph

In [5]:
graph = Graph()

graph.add('0')
graph.add('1')
graph.add('2')
graph.add('3')
graph.add('4')
graph.connect('0', '1', 5)
graph.connect('0', '2', 3)
graph.connect('0', '4', 2)
graph.connect('1', '2', 2)
graph.connect('1', '3', 6)
graph.connect('2', '1', 1)
graph.connect('2', '3', 2)
graph.connect('4', '1', 6)
graph.connect('4', '2', 10)
graph.connect('4', '3', 4)

graph.printGraph()

graph.remove('1')

graph.printGraph()

Vertices:
  Vertex In Out
0      0  0   3
1      1  3   2
2      2  3   2
3      3  3   0
4      4  1   3
--------------------
Edges:
  From To Weight
0    0  1    5.0
1    0  2    3.0
2    0  4    2.0
3    1  2    2.0
4    1  3    6.0
5    2  1    1.0
6    2  3    2.0
7    4  1    6.0
8    4  2   10.0
9    4  3    4.0
Vertices:
  Vertex In Out
0      0  0   2
1      2  2   1
2      3  2   0
3      4  1   2
--------------------
Edges:
  From To Weight
0    0  2    3.0
1    0  4    2.0
2    2  3    2.0
3    4  2   10.0
4    4  3    4.0


# Approximate Inference

In [6]:
# Generate new samples
def sampling(X=[], P=[], size=1):
  genList = np.random.uniform(0, 1, size)
  n = len(P)
  lowerP = np.dot(P, np.tri(n, n, -1).T)
  clf_matrix = genList.reshape(len(genList), 1) - lowerP.reshape(1, len(lowerP))
  clf_idx = np.apply_along_axis(lambda x: len(x[x >= 0]) - 1, 1, clf_matrix)

  return np.array(X)[clf_idx]

In [7]:
print(sampling(X=['Y','TB','K','G'], P=[0.1,0.3,0.4,0.2], size=5))

['G' 'K' 'Y' 'TB' 'G']
